# Quick Usage: The "Hello World" of Causality

In this example, we will:
1. Generate a small synthetic dataset (5 variables).
2. Train a lightweight TD2C model on the fly.
3. Use it to infer causal links in a new time series.

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from imblearn.ensemble import BalancedRandomForestClassifier

# Import TD2C modules
from td2c.data_generation.builder import TSBuilder
from td2c.descriptors import DataLoader
from td2c.benchmark import D2CWrapper

# 1. Generate Synthetic Data (Small scale for demo)
print("Generating synthetic training data...")
builder = TSBuilder(
    n_variables=3, 
    maxlags=2, 
    observations_per_time_series=100,
    time_series_per_process=5, # Small number for speed
    processes_to_use=[1],      # Just one generative process
    verbose=False
)
builder.build()

# 2. Prepare Data for Training
loader = DataLoader(n_variables=3, maxlags=2)
loader.from_tsbuilder(builder)

# We wrap the training logic here
# In a real scenario, you would load a pre-trained model
from td2c.descriptors import D2C

print("Computing descriptors (Feature Engineering)...")
d2c_engine = D2C(
    observations=loader.get_observations(),
    dags=loader.get_dags(),
    n_variables=3,
    maxlags=2,
    n_jobs=1,
    full=True,
    dynamic=True,
    mb_estimator="ts"
)
d2c_engine.initialize()
df_train = d2c_engine.get_descriptors_df()

# 3. Train the Classifier
print("Training Classifier...")
X_train = df_train.drop(columns=["graph_id", "edge_source", "edge_dest", "is_causal"])
y_train = df_train["is_causal"]

model = BalancedRandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)
print("✅ Model Trained!")

### Inference
Now we generate a **new, unseen** time series and ask our model to find the causal graph.

In [ ]:
# Generate a new test sample
test_builder = TSBuilder(n_variables=3, maxlags=2, time_series_per_process=1, processes_to_use=[1], verbose=False)
test_builder.build()
obs_test = test_builder.get_generated_observations()[1][0] # Get the raw array

# Run TD2C Inference
# We pass the trained model to the Wrapper
d2c_wrapper = D2CWrapper(
    ts_list=[obs_test], # It expects a list
    model=model,
    n_variables=3,
    maxlags=2,
    mb_estimator="ts"
)

d2c_wrapper.run()
results = d2c_wrapper.get_causal_dfs()[0]

# Display Results
print("\nInferred Causal Links (Probability > 0.5):")
print(results[results['probability'] > 0.5][['from', 'to', 'probability']])